# Notebook 02 - Data Cleaning

## Objective

The purpose of this notebook is to clean, standardize, and consolidate the raw CSV files from the Bronze layer into a single, structured dataset suitable for further analysis.
In this notebook, we:

- Load and merge all original CSV files
- Remove fully duplicated rows while keeping legitimate repeated flow IDs
- Parse and validate timestamps
- Inspect and handle missing values
- Verify and correct data types across all features
- Detect outliers without removing them, preserving attack-related anomalies
- Export the cleaned dataset to the Silver layer for downstream EDA and modeling

This notebook focuses on preparing the data while ensuring no attack-related information is lost. No feature engineering or modeling is performed here.

# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import os
from pathlib import Path

# load and Unify CSV into a DF

In [2]:
# Path to Bronze layer
bronze_path = Path("../data/bronze")

# List all CSV files
csv_files = list(bronze_path.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files in Bronze.")

# List to store dataframes
df_list = []
failed_files = []

for file in csv_files:
    try:
        print(f"Loading: {file.name} ...")
        df = pd.read_csv(file)
        
        # Add a column with the source file name (optional but very useful)
        df["source_file"] = file.name
        
        df_list.append(df)

    except Exception as e:
        print(f"ERROR loading {file.name}: {e}")
        failed_files.append(file.name)

# Concatenate all successfully loaded DataFrames
if df_list:
    df_all = pd.concat(df_list, ignore_index=True)
    print("\n Successfully merged all CSV files.")
    print(f"Total rows: {df_all.shape[0]:,}")
    print(f"Total columns: {df_all.shape[1]}")

else:
    print("\n No CSV files were successfully loaded.")

# Report failed files, if any
if failed_files:
    print("\n The following files could NOT be loaded:")
    for f in failed_files:
        print(" -", f)
else:
    print("\nAll files loaded successfully!")


Found 15 CSV files in Bronze.
Loading: Benign Traffic.csv ...
Loading: DDoS ICMP Flood.csv ...
Loading: DDoS UDP Flood.csv ...
Loading: DoS ICMP Flood.csv ...
Loading: DoS TCP Flood.csv ...
Loading: DoS UDP Flood.csv ...
Loading: MITM ARP Spoofing.csv ...
Loading: MQTT DDoS Publish Flood.csv ...
Loading: MQTT DoS Connect Flood.csv ...
Loading: MQTT DoS Publish Flood.csv ...
Loading: MQTT Malformed.csv ...
Loading: Recon OS Scan.csv ...
Loading: Recon Ping Sweep.csv ...
Loading: Recon Port Scan.csv ...
Loading: Recon Vulnerability Scan.csv ...

 Successfully merged all CSV files.
Total rows: 3,385,313
Total columns: 86

All files loaded successfully!


In [3]:
df_all.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Attack Name,Label,source_file
0,34.173.20.6-192.168.137.234-443-58644-6,34.173.20.6,443,192.168.137.234,58644,6,14/09/2023 09:24:26 AM,14746364,6,0,...,0.0,0.0,0.0,14681116.0,0.000000,14681116.0,14681116.0,Benign Traffic,0,Benign Traffic.csv
1,34.173.20.6-192.168.137.234-443-58646-6,34.173.20.6,443,192.168.137.234,58646,6,14/09/2023 09:24:41 AM,60102110,18,0,...,0.0,219123.0,219123.0,14957706.0,190543.248665,15061923.0,14672051.0,Benign Traffic,0,Benign Traffic.csv
2,52.40.210.103-192.168.137.40-18665-60378-6,52.40.210.103,18665,192.168.137.40,60378,6,14/09/2023 09:24:14 AM,90106769,9,0,...,0.0,582529.0,582529.0,29841412.0,319090.538141,30049066.0,29473997.0,Benign Traffic,0,Benign Traffic.csv
3,10.0.0.3-10.0.0.254-44505-1883-6,10.0.0.3,44505,10.0.0.254,1883,6,14/09/2023 09:24:15 AM,119998980,123,123,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign Traffic,0,Benign Traffic.csv
4,10.0.0.7-10.0.0.254-45727-1883-6,10.0.0.7,45727,10.0.0.254,1883,6,14/09/2023 09:24:15 AM,119998539,123,123,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign Traffic,0,Benign Traffic.csv


In [4]:
df_all.tail()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Attack Name,Label,source_file
3385308,192.168.137.174-192.168.137.48-9000-34233-6,192.168.137.174,9000,192.168.137.48,34233,6,12/09/2023 12:32:05 PM,210290,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recon Vulnerability Scan,1,Recon Vulnerability Scan.csv
3385309,192.168.137.221-192.168.137.48-23502-45314-6,192.168.137.221,23502,192.168.137.48,45314,6,12/09/2023 12:32:08 PM,202834,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recon Vulnerability Scan,1,Recon Vulnerability Scan.csv
3385310,192.168.137.174-192.168.137.48-5440-34233-6,192.168.137.174,5440,192.168.137.48,34233,6,12/09/2023 12:32:05 PM,208211,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recon Vulnerability Scan,1,Recon Vulnerability Scan.csv
3385311,192.168.137.140-192.168.137.48-9003-49424-6,192.168.137.140,9003,192.168.137.48,49424,6,12/09/2023 12:28:54 PM,132317,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recon Vulnerability Scan,1,Recon Vulnerability Scan.csv
3385312,192.168.137.237-192.168.137.48-1839-62935-6,192.168.137.237,1839,192.168.137.48,62935,6,12/09/2023 12:28:41 PM,210121,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Recon Vulnerability Scan,1,Recon Vulnerability Scan.csv


# Detect and Delete Duplicates

In [5]:
# Inspect duplicates before removal ---
print("Checking for duplicates...\n")

total_rows = df_all.shape[0]
duplicate_rows = df_all.duplicated().sum()

print(f"Total rows before cleaning: {total_rows:,}")
print(f"Duplicate rows found: {duplicate_rows:,}")

# Optional: check if Flow ID has duplicates
if "Flow ID" in df_all.columns:
    flow_id_dups = df_all["Flow ID"].duplicated().sum()
    print(f"Duplicate Flow ID values: {flow_id_dups:,}")
else:
    print("Column 'Flow ID' not found.")

# Backup before cleaning
df_backup = df_all.copy()

# Remove duplicates
df_all = df_all.drop_duplicates().reset_index(drop=True) # Reset index after dropping duplicates to keep it clean

print("\n Duplicates removed.")
print(f"Total rows after cleaning: {df_all.shape[0]:,}")
print(f"Rows removed: {total_rows - df_all.shape[0]:,}")


Checking for duplicates...

Total rows before cleaning: 3,385,313
Duplicate rows found: 109,180
Duplicate Flow ID values: 2,330,072

 Duplicates removed.
Total rows after cleaning: 3,276,133
Rows removed: 109,180


# Review null columns

In [6]:
print("Checking for null values...\n")

null_counts = df_all.isnull().sum()
total_rows = len(df_all)

null_summary = pd.DataFrame({
    'null_count': null_counts,
    'null_percentage': (null_counts / total_rows) * 100
})

# Mostrar solo columnas con nulos
null_summary = null_summary[null_summary['null_count'] > 0]

if null_summary.empty:
    print("No null values found in the dataset")
else:
    display(null_summary.sort_values('null_count', ascending=False))


Checking for null values...

No null values found in the dataset


# Explore column data type, standarize and clean it

In [7]:
pd.set_option('display.max_rows', None) # show all rows
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.width', None)   # set no limit on display width
pd.set_option('display.max_colwidth', None) # set no limit on column width

print("Data types of all columns:\n")
print(df_all.dtypes)

Data types of all columns:

Flow ID                        object
Src IP                         object
Src Port                        int64
Dst IP                         object
Dst Port                        int64
Protocol                        int64
Timestamp                      object
Flow Duration                   int64
Total Fwd Packet                int64
Total Bwd packets               int64
Total Length of Fwd Packet    float64
Total Length of Bwd Packet    float64
Fwd Packet Length Max         float64
Fwd Packet Length Min         float64
Fwd Packet Length Mean        float64
Fwd Packet Length Std         float64
Bwd Packet Length Max         float64
Bwd Packet Length Min         float64
Bwd Packet Length Mean        float64
Bwd Packet Length Std         float64
Flow Bytes/s                  float64
Flow Packets/s                float64
Flow IAT Mean                 float64
Flow IAT Std                  float64
Flow IAT Max                  float64
Flow IAT Min          

In [8]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

In [9]:
df_all["Timestamp"].head()

0    14/09/2023 09:24:26 AM
1    14/09/2023 09:24:41 AM
2    14/09/2023 09:24:14 AM
3    14/09/2023 09:24:15 AM
4    14/09/2023 09:24:15 AM
Name: Timestamp, dtype: object

In [11]:
print("Standardizing data types...\n")

# 1. Convert Timestamp to datetime ---
if 'Timestamp' in df_all.columns:
    print("Converting Timestamp to datetime...")
    df_all['Timestamp'] = pd.to_datetime(df_all['Timestamp'], errors='coerce')
else:
    print("Timestamp column not found.")

# 2. Replace 'Infinity' or invalid strings in numeric columns
print("\nCleaning invalid numeric values (e.g., 'Infinity', 'NaN' strings)...")

for col in df_all.columns:
    if df_all[col].dtype == object:
        # skip columns that must remain text
        if col in ["Flow ID", "Src IP", "Dst IP", "Attack Name", "source_file"]:
            continue
        
        # try converting
        df_all[col] = pd.to_numeric(df_all[col], errors='ignore')

# 3. Ensure all numeric columns are float64 ---
numeric_cols = df_all.select_dtypes(include=['int64', 'float64']).columns

print("\nConverting all numeric columns to float64 for consistency...")
df_all[numeric_cols] = df_all[numeric_cols].astype('float64')

# 4. Ensure Label is int64
if "Label" in df_all.columns:
    df_all["Label"] = df_all["Label"].astype("int64")

print("\nData type standardization completed!")
pd.set_option('display.max_rows', None) # show all rows
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.width', None)   # set no limit on display width
pd.set_option('display.max_colwidth', None) # set no limit on column width
print(df_all.dtypes)


Standardizing data types...

Converting Timestamp to datetime...

Cleaning invalid numeric values (e.g., 'Infinity', 'NaN' strings)...

Converting all numeric columns to float64 for consistency...

Data type standardization completed!
Flow ID                               object
Src IP                                object
Src Port                             float64
Dst IP                                object
Dst Port                             float64
Protocol                             float64
Timestamp                     datetime64[ns]
Flow Duration                        float64
Total Fwd Packet                     float64
Total Bwd packets                    float64
Total Length of Fwd Packet           float64
Total Length of Bwd Packet           float64
Fwd Packet Length Max                float64
Fwd Packet Length Min                float64
Fwd Packet Length Mean               float64
Fwd Packet Length Std                float64
Bwd Packet Length Max                float64


In [12]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

# Outliers

In [13]:
num_cols = df_all.select_dtypes(include=['float64', 'int64']).columns

In [16]:
outlier_report = {}

for col in num_cols:
    Q1 = df_all[col].quantile(0.25)
    Q3 = df_all[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = ((df_all[col] < lower) | (df_all[col] > upper)).sum()

    outlier_report[col] = outliers

outlier_df = (
    pd.DataFrame.from_dict(outlier_report, orient='index', columns=['outliers'])
    .sort_values(by='outliers', ascending=False)
)

In [17]:
pd.set_option('display.max_rows', None) # show all rows
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.width', None)   # set no limit on display width
pd.set_option('display.max_colwidth', None) # set no limit on column width

display(outlier_df)

,outliers
SYN Flag Count,816905
Down/Up Ratio,725389
Idle Std,671173
Active Max,649703
Active Mean,641972
Fwd IAT Std,581863
Flow IAT Std,581796
Active Std,529133
Idle Mean,442191
Flow Bytes/s,423553


The detected outliers are not errors but rather core indicators of attack behavior in the dataset. Most occur in features such as TCP flag counts, packet rates, IAT timings, and packet size metrics—dimensions where attacks naturally differ from benign traffic. Therefore, outliers must not be removed, as they represent the malicious patterns the model needs to learn. They should only be analyzed to confirm their relationship with attack behavior. Normalization will be applied later without eliminating these extreme values.

# Compare outliers Benign vs Attack

In [18]:
num_cols = df_all.select_dtypes(include=['float64', 'int64']).columns

outlier_by_class = {}

for col in num_cols:
    Q1 = df_all[col].quantile(0.25)
    Q3 = df_all[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outlier_by_class[col] = df_all.groupby("Label")[col].apply(
        lambda x: ((x < lower) | (x > upper)).sum()
    )

outlier_by_class_df = pd.DataFrame(outlier_by_class).T
display(outlier_by_class_df)

Label,0,1
Src Port,0,0
Dst Port,0,0
Protocol,0,0
Flow Duration,0,0
Total Fwd Packet,4768,2923
Total Bwd packets,25920,255172
Total Length of Fwd Packet,30717,283373
Total Length of Bwd Packet,28161,260027
Fwd Packet Length Max,30717,283373
Fwd Packet Length Min,3257,20052


# Load clean data to silver

In [21]:
# Define output path
silver_path = Path("../data/silver")
silver_path.mkdir(parents=True, exist_ok=True)

# Save cleaned dataset
output_file = silver_path / "data_merge_and_cleaned.csv"

df_all.to_csv(output_file, index=False)

print(f"Cleaned dataset saved successfully at: {output_file}")


Cleaned dataset saved successfully at: ..\data\silver\data_merge_and_cleaned.csv


# Summary of this stage

In this stage, we consolidated all CSV files from the Bronze layer into a single dataset and performed essential cleaning tasks, including duplicate removal, timestamp parsing, null-value inspection, type validation, and outlier analysis. The results showed that the dataset contains many extreme values, but these are not noise—they are key behavioral indicators of attacks such as DoS, DDoS, Reconnaissance, MQTT floods, and MITM. Therefore, outliers were preserved rather than removed. The cleaned dataset is now stored in the Silver layer and is ready for the exploratory data analysis (EDA) phase, where deeper insights and feature behavior will be examined.